In [1]:
import lpips
import torch
import torchvision.transforms as transforms
from skimage.metrics import structural_similarity as ssim_skimage
from skimage.metrics import peak_signal_noise_ratio as psnr
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import re
import numpy as np

from torchmetrics.image import UniversalImageQualityIndex
from torchmetrics.image import VisualInformationFidelity

Configuration

In [2]:
distorted_images_dir = "Distortions_v2"
csv_output_dir = "CSVs"
chart_output_dir = "Charts"

recalculate_values = False

Measuring values and saving to csv file

In [3]:
df = pd.read_csv(csv_output_dir + "/distortion_results.csv")
scene_folder = "Degus"
distortion_folder = "Blur"
distortion_value = 1
df_i = df[
                    (df["Picture"] == scene_folder) &
                    (df["DistortionType"] == distortion_folder) &
                    (df["DistortionValue"] == distortion_value)
                ]
print(df_i)

FileNotFoundError: [Errno 2] No such file or directory: 'CSVs/distortion_results.csv'

In [ ]:
from scipy import signal

def cal_ssim(img1, img2):

    K = [0.01, 0.03]
    L = 255
    kernelX = cv2.getGaussianKernel(11, 1.5)
    window = kernelX * kernelX.T

    M,N = np.shape(img1)

    C1 = (K[0]*L)**2
    C2 = (K[1]*L)**2
    img1 = np.float64(img1)
    img2 = np.float64(img2)

    mu1 = signal.convolve2d(img1, window, 'valid')
    mu2 = signal.convolve2d(img2, window, 'valid')

    mu1_sq = mu1*mu1
    mu2_sq = mu2*mu2
    mu1_mu2 = mu1*mu2


    sigma1_sq = signal.convolve2d(img1*img1, window, 'valid') - mu1_sq
    sigma2_sq = signal.convolve2d(img2*img2, window, 'valid') - mu2_sq
    sigma12 = signal.convolve2d(img1*img2, window, 'valid') - mu1_mu2

    ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))
    mssim = np.mean(ssim_map)
    return mssim,ssim_map

In [ ]:

# Initialize LPIPS model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_fn = lpips.LPIPS(net='alex').to(device)

# Transform: convert image to tensor and normalize to [-1, 1]
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Optional: adjust to match your data
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # LPIPS expects input in [-1, 1]
])

results = []
df = None
if not recalculate_values:
    try:
        df = pd.read_csv(csv_output_dir + "/distortion_results.csv")
    except (FileNotFoundError, pd.errors.EmptyDataError):
        print('Results file not found. Will calculate all values.')
        recalculate_values = True

uiqi_torch = UniversalImageQualityIndex()
vif_torch = VisualInformationFidelity()
# Loop over picture folders (Degus, Hotdog, etc.)
for scene_folder in os.listdir(distorted_images_dir):
    print(scene_folder)
    scene_path = os.path.join(distorted_images_dir, scene_folder)
    if not os.path.isdir(scene_path):
        continue

    # Loop over distortion type folders (Blur, Noise, etc.)
    for distortion_folder in os.listdir(scene_path):
        print(distortion_folder)
        distortion_path = os.path.join(scene_path, distortion_folder)
        if not os.path.isdir(distortion_path):
            continue

        # Find ref image (ends with 0.bmp and is only .bmp file)
        reference_image_path = None
        for file in os.listdir(distortion_path):
            if file.endswith('_0.bmp'):
                reference_image_path = os.path.join(distortion_path, file)
                break
        if reference_image_path is None:
            print(f"No reference image found in {scene_path}")
            continue
        ref_img_lpips = transform(Image.open(reference_image_path).convert('RGB')).unsqueeze(0).to(device)
        ref_img_ssim = cv2.cvtColor(cv2.imread(reference_image_path), cv2.COLOR_BGR2RGB)
        ref_img_psnr = cv2.imread(reference_image_path)
        

        normalized_ref_img = (cv2.imread(reference_image_path).astype(np.float32) / 127.5) - 1.0 

        tensor_ref_img = torch.from_numpy(normalized_ref_img).permute(2, 0, 1)



        uiqi_torch.reset()
        vif_torch.reset()
        for file in os.listdir(distortion_path):
            print(file)
            if not file.endswith('.bmp') and not file.endswith('.png'):
                continue

            #extract distortion value
            distortion_value = -1
            match = re.search(r'^([^_]+)_([^_]+)_(.*)\.(png|bmp)$', file)
            try:
                distortion_value = float(match.group(3).replace('_', '.'))
            except:
                print(f"No distortion value found in {file}")

            # Load and preprocess distorted image
            distorted_image_path = os.path.join(distortion_path, file)
            df_i = None
            if not recalculate_values:
                df_i = df[
                    (df["Picture"] == scene_folder) &
                    (df["DistortionType"] == distortion_folder) &
                    (df["DistortionValue"] == distortion_value)
                ]
            
            normalized_distorted_img = (cv2.imread(distorted_image_path).astype(np.float32) / 127.5) - 1.0
            tensor_distorted_img = torch.from_numpy(normalized_distorted_img).permute(2, 0, 1)

            # Compute LPIPS (lpips) distance
            if recalculate_values or df_i[df_i["Method"] == "LPIPS"].empty:
                dist_img_lpips = transform(Image.open(distorted_image_path).convert('RGB')).unsqueeze(0).to(device)
                with torch.no_grad():
                    distance_lpips = loss_fn(ref_img_lpips, dist_img_lpips).item()

                results.append({
                    'Picture': scene_folder,
                    'DistortionType': distortion_folder,
                    'DistortionValue': distortion_value,
                    'Filename': file,
                    'Metric': 'LPIPS',
                    'Method': 'LPIPS_lpips',
                    'Value': distance_lpips
                })
            else:
                print(f"Skipping {file}, LPIPS is already calculated")

            # Compute SSIM
            if recalculate_values or df_i[df_i["Method"] == "SSIM"].empty:
                dist_img_ssim  = cv2.cvtColor(cv2.imread(distorted_image_path), cv2.COLOR_BGR2RGB)
                distance_ssim, _ = ssim_skimage(
                    ref_img_ssim,
                    dist_img_ssim,
                    channel_axis=2,
                    full=True
                )

                results.append({
                    'Picture': scene_folder,
                    'DistortionType': distortion_folder,
                    'DistortionValue': distortion_value,
                    'Filename': file,
                    'Metric': 'SSIM',
                    'Method': 'SSIM_skimage',
                    'Value': distance_ssim
                })
            else:
                print(f"Skipping {file}, SSIM is already calculated")

            if recalculate_values or df_i[df_i["Method"] == "PSNR"].empty:
                # Compute PSNR value
                dist_img_psnr  = cv2.imread(distorted_image_path)
                distance_psnr = psnr(ref_img_psnr, dist_img_psnr, data_range=255)
                results.append({
                    'Picture': scene_folder,
                    'DistortionType': distortion_folder,
                    'DistortionValue': distortion_value,
                    'Filename': file,
                    'Metric': 'PSNR',
                    'Method': 'PSNR_skimage',
                    'Value': distance_psnr
                })
            else:
                print(f"Skipping {file}, PSNR is already calculated")

            # Compute UIQI
            if recalculate_values or df_i[df_i["Method"] == "UIQI"].empty:
                # Compute UIQI value
                distance_UIQI = uiqi_torch(tensor_ref_img.expand(1,-1,-1,-1), tensor_distorted_img.expand(1,-1,-1,-1))
                results.append({
                    'Picture': scene_folder,
                    'DistortionType': distortion_folder,
                    'DistortionValue': distortion_value,
                    'Filename': file,
                    'Metric': 'UIQI',
                    'Method': 'UIQI_torch',
                    'Value': distance_UIQI.item()
                })
            else:
                print(f"Skipping {file}, UIQI is already calculated")

            # Compute VIF
            if recalculate_values or df_i[df_i["Method"] == "VIF"].empty:
                # Compute VIF value
                distance_VIF = vif_torch(tensor_ref_img.expand(1,-1,-1,-1), tensor_distorted_img.expand(1,-1,-1,-1))
                results.append({
                    'Picture': scene_folder,
                    'DistortionType': distortion_folder,
                    'DistortionValue': distortion_value,
                    'Filename': file,
                    'Metric': 'VIF',
                    'Method': 'VIF_torch',
                    'Value': distance_VIF.item()
                })
            else:
                print(f"Skipping {file}, VIF is already calculated")


# Save all results to CSV
df_new = pd.DataFrame(results)
os.makedirs(csv_output_dir, exist_ok=True)
csv_path = os.path.join(csv_output_dir, 'distortion_results.csv')
file_exists = os.path.isfile(csv_path)
df_new.to_csv(csv_path, mode='a', header=not file_exists, index=False)
print("✅ Results saved to distortion_results.csv")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: C:\Users\piotr\AppData\Roaming\Python\Python312\site-packages\lpips\weights\v0.1\alex.pth
Degus
Blur
Degus_blur_0.bmp
Degus_blur_1.png
Degus_blur_10.png
Degus_blur_12_5.png
Degus_blur_15.png
Degus_blur_2.png
Degus_blur_3.png
Degus_blur_5.png
Degus_blur_7_5.png
Brightness
Degus_brightness_0.bmp
Degus_brightness_1_02.png
Degus_brightness_1_05.png
Degus_brightness_1_1.png
Degus_brightness_1_2.png
Degus_brightness_1_5.png
Degus_brightness_1_75.png
Degus_brightness_2.png
Degus_brightness_2_25.png
Circularblur
Degus_circularBlur_0.bmp
Degus_circularBlur_1.png
Degus_circularBlur_10.png
Degus_circularBlur_2.png
Degus_circularBlur_3.png
Degus_circularBlur_4.png
Degus_circularBlur_5.png
Degus_circularBlur_7_5.png
Dilation
Degus_dilation_0.bmp
Degus_dilation_1.png
Degus_dilation_2.png
Degus_dilation_3.png
Degus_dilation_4.png
Degus_dilation_5.png
Hole
Degus_hole_0.bmp
Degus_hole_0_01.png
Degus_hole_0_05.pn

# NULCEAR EMERGENCY: USE IF METRICS ARE FUCKED UP

In [ ]:
df = pd.read_csv(csv_path)

# Function to extract the float from a string like "tensor(6.8433)"
def strip_tensor_string(val):
    if isinstance(val, str) and val.startswith('tensor(') and val.endswith(')'):
        try:
            return float(val[len('tensor('):-1])
        except ValueError:
            return val  # leave unchanged if it can't be converted
    return val  # leave unchanged if not a tensor string

# Apply the function to all cells
csv_path_cleaned = os.path.join(csv_output_dir, 'distortion_results_cleaned.csv')
df = df.applymap(strip_tensor_string)

df.loc[df.index % 5 == 4, 'Metric'] = 'VIF'
df.loc[df.index % 5 == 4, 'Method'] = 'VIF_Torch'

# Optional: Save to a new CSV
df.to_csv(csv_path_cleaned, index=False)

C:\Users\piotr\AppData\Local\Temp\ipykernel_17436\3478999076.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(strip_tensor_string)


Creating charts

In [17]:
# === Load CSV ===
df = pd.read_csv(csv_output_dir + "/distortion_results_cleaned.csv")

# Add synthetic LPIPS=0 for level 0 reference images
reference_rows = []
for picture in df['Picture'].unique():
    for distortion in df['DistortionType'].unique():
        value = 0
        if(distortion == 'Brightness' or distortion == 'Saturation'):
            value = 1
        reference_rows.append({
            'Picture': picture,
            'DistortionType': distortion,
            'DistortionValue': value,
            'Filename': 'image_0.bmp',
            'Method': 'LPIPS',
            'Value': 0.0
        })

df = pd.concat([df, pd.DataFrame(reference_rows)], ignore_index=True)
# === Plotting ===
os.makedirs(chart_output_dir, exist_ok=True)

for method in df['Metric'].unique():
    method_df = df[df['Metric'] == method]
    for distortion in df['DistortionType'].unique():
        plt.figure(figsize=(8, 5))
        distortion_df = method_df[method_df['DistortionType'] == distortion]

        for picture in sorted(distortion_df['Picture'].unique()):
            picture_df = distortion_df[distortion_df['Picture'] == picture]
            picture_df = picture_df.sort_values('DistortionValue')

            # Build y and x values
            x = picture_df['DistortionValue']
            y = picture_df['Value']
            plt.plot(x, y, marker='o', label=picture)

        plt.title(f'{method} vs Distortion Value – {distortion}')
        plt.xlabel('Distortion Parameter')
        plt.ylabel(f'{method} Distance')
        plt.legend(title='Picture', fontsize='small')
        plt.grid(True)
        plt.tight_layout()
        os.makedirs(f'{chart_output_dir}/{method}', exist_ok=True)
        plt.savefig(f'{chart_output_dir}/{method}/{distortion}_{method}_chart.png')
        plt.close()

print("✅ All charts saved")

C:\Users\piotr\AppData\Local\Temp\ipykernel_17436\541571912.py:42: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend(title='Picture', fontsize='small')


✅ All charts saved


Creating group chart

In [12]:
df = pd.read_csv(csv_output_dir + "/distortion_results.csv")

# Keep rows with the highest DistortionValue for each Picture & DistortionType
idx = df.groupby(['Picture', 'DistortionType'])['DistortionValue'].idxmax()
df_max_distortions = df.loc[idx]
df_max_distortions['DistortionLabel'] = df_max_distortions.apply(lambda row: f"{row['DistortionType']} ({row['DistortionValue']})", axis=1)

# Pivot for grouped bar chart: index=DistortionType, columns=Picture, values=LPIPS
pivot_df = df_max_distortions.pivot(index='DistortionLabel', columns='Picture', values='Value')

# Plot
pivot_df.plot(kind='bar', figsize=(10, 6))

plt.ylabel('LPIPS Distance')
plt.title('LPIPS at Max Distortion per Picture per Distortion Type')
plt.xticks(rotation=45)
plt.legend(title='Picture')
plt.tight_layout()
plt.savefig(f'{chart_output_dir}/lpips_distortion_max.png')
plt.close()


In [ ]:
# Load the dataset if needed (skip if already loaded)
# df = pd.read_csv(csv_output_dir + "/distortion_results.csv")

# List of images to process
pictures = ['Degus', 'Hotdog', 'Ziemniaki']

for picture in pictures:
    df_pic = df[df['Picture'] == picture]

    # Get max distortion per distortion type and metric
    idx = df_pic.groupby(['DistortionType', 'Metric'])['DistortionValue'].idxmax()
    df_pic_max = df_pic.loc[idx]

    # Create label like "Blur (5.0)"
    df_pic_max['DistortionLabel'] = df_pic_max.apply(
        lambda row: f"{row['DistortionType']} ({row['DistortionValue']})", axis=1
    )

    # Pivot: rows = distortion labels, columns = metrics, values = metric values
    pivot_df = df_pic_max.pivot(index='DistortionLabel', columns='Metric', values='Value')

    # Ensure column order is LPIPS, PSNR, SSIM (if present)
    desired_order = ['LPIPS', 'PSNR', 'SSIM']
    pivot_df = pivot_df[[col for col in desired_order if col in pivot_df.columns]]

    # Normalize each metric independently (column-wise)
    normalized_df = pivot_df / pivot_df.max()

    # Plot
    ax = normalized_df.plot(kind='bar', figsize=(10, 6))
    plt.ylabel('Normalized Metric Value')
    plt.title(f'Normalized Metrics at Max Distortion per Distortion Type for "{picture}"')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Metric')
    plt.tight_layout()
    plt.savefig(f"{chart_output_dir}/{picture.lower()}_normalized_metrics.png")
    plt.close()

In [ ]:
# List of pictures to average
pictures = ['Degus', 'Hotdog', 'Ziemniaki']
all_rows = []

# Collect max distortion rows for each image
for picture in pictures:
    df_pic = df[df['Picture'] == picture]
    idx = df_pic.groupby(['DistortionType', 'Method'])['DistortionValue'].idxmax()
    df_pic_max = df_pic.loc[idx].copy()
    df_pic_max['Picture'] = picture  # Ensure Picture label is preserved
    all_rows.append(df_pic_max)

# Combine all into one DataFrame
df_all = pd.concat(all_rows)

# Add DistortionLabel for grouping
df_all['DistortionLabel'] = df_all.apply(
    lambda row: f"{row['DistortionType']} ({row['DistortionValue']})", axis=1
)

# Pivot: rows = (DistortionLabel, Picture), columns = Metric, values = Value
pivot = df_all.pivot_table(index=['DistortionLabel', 'Picture'], columns='Method', values='Value')

# Average across pictures per DistortionLabel
pivot_avg = pivot.groupby('DistortionLabel').mean()

# Ensure consistent metric order
desired_order = ['LPIPS', 'PSNR', 'SSIM']
pivot_avg = pivot_avg[[col for col in desired_order if col in pivot_avg.columns]]

# Normalize each metric individually
pivot_avg_norm = pivot_avg / pivot_avg.max()

# Plot
ax = pivot_avg_norm.plot(kind='bar', figsize=(10, 6))
plt.ylabel('Normalized Average Metric Value')
plt.title('Normalized Average Metrics at Max Distortion per Distortion Type (All Images)')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Metric')
plt.tight_layout()
plt.savefig(f"{chart_output_dir}/average_normalized_metrics.png")
plt.close()

: 